In [ ]:
!pip install transformers
!pip install nltk
!pip install git+https://github.com/openai/whisper.git 
!python -m pip install -U yt-dlp
!pip install evaluate
!pip install rouge_score
!pip install gradio
!pip install huggingface_hub
# !pip install youtube_dl
# !pip install imageio_ffmpeg

In [ ]:
import os
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')
from tqdm import tqdm
import pandas as pd
from pathlib import Path
import whisper
from moviepy.video.io.VideoFileClip import VideoFileClip
from transformers import pipeline
import evaluate
import shutil
import pandas as pd
pd.set_option('display.max_colwidth',None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
folder_to_make = ['summary','texts','audios','videos']
for folder in folder_to_make:
    os.mkdir(rf'/content/{folder}')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to the source file
source_file = ["/content/drive/MyDrive/ann1.mp4","/content/drive/MyDrive/ann2.mp4"]

# Path to the destination folder
destination_folder = "/content/videos/"

# Move the file to the destination folder
for video in source_file:
  shutil.copy(video, destination_folder)


In [ ]:
# video_urls = ['https://youtu.be/zLP6oT3uqV8','https://youtu.be/aNIVHAtqnUM']

# for each in video_urls:
#   os.chdir('/content/videos/')
#   !yt-dlp {each}
# os.chdir('/content/')

In [ ]:
def video_to_audio(videos,save_path = "/content/audios"):
    video_to_audio_dir = Path(save_path)
    if type(videos) == str:
        videos = [videos]
    elif type(videos) == list:
        pass
    else:
        print('Invalid Input Path')
        
    for i in tqdm(videos):
        video = VideoFileClip(fr"{i}")

        if '.mp4' in i:
          name = i.split('/')[-1].split('.mp4')[0]
        elif '.webm' in i:
          name = i.split('/')[-1].split('.webm')[0]
        else:
          pass

        audio = video.audio
        audio.write_audiofile(f'{video_to_audio_dir/name}.mp3')
        audio.close()

In [ ]:
videos = os.listdir('/content/videos')
[video_to_audio(f'/content/videos/{video}') for video in videos]

  0%|          | 0/1 [00:00<?, ?it/s]

MoviePy - Writing audio in /content/audios/ann2.mp3



100%|██████████| 1/1 [00:01<00:00,  1.86s/it]


MoviePy - Done.


  0%|          | 0/1 [00:00<?, ?it/s]

MoviePy - Writing audio in /content/audios/ann1.mp3



100%|██████████| 1/1 [00:01<00:00,  1.89s/it]

MoviePy - Done.


[None, None]

In [ ]:
model_medium = whisper.load_model('medium',device='cuda:0')

100%|██████████████████████████████████████| 1.42G/1.42G [00:08<00:00, 179MiB/s]


In [ ]:
def audio_to_text(audios,save_path = "/content/texts/"):
    audio_to_text_dir = Path(save_path)

    if type(audios) == str:
        audios = [audios]
    elif type(audios) == list:
        pass
    else:
        print('Invalid Input Path')
    
    for i in tqdm(audios):
        name = audio_to_text_dir.as_posix()
        name += '/' + i.split('/')[-1].split('.mp3')[0]
        path_for_writing = Path(name)
        print(path_for_writing)
        result_medium = model_medium.transcribe(i)['text']
        with open(path_for_writing,'w') as f:
            f.write(result_medium)

In [ ]:
audios_path = os.listdir('/content/audios')
[audio_to_text(f'/content/audios/{audio}') for audio in audios_path]

  0%|          | 0/1 [00:00<?, ?it/s]

/content/texts/ann1


  0%|          | 0/1 [00:00<?, ?it/s]

/content/texts/ann2


100%|██████████| 1/1 [00:11<00:00, 11.69s/it]


[None, None]

In [ ]:
def get_file_content(filepath):
    filepath = Path(filepath)
    file = open(filepath,"r")
    return file.read().strip()

In [ ]:
text_path = os.listdir('/content/texts')
text_path = ['/content/texts/'+text for text in text_path]

In [ ]:
generated_texts = [get_file_content(path) for path in text_path]
generated_texts

["Hello. Hello. Hi, Akshay. How are you? I'm good. So today we're going to talk. Yes, yes, continue. Today we're going to talk about artificial neural networks. That is ANN. ANN has become the cornerstone of the world. The ability to mimic the human brain's information processing has lead to breakthroughs in fields such as computer vision, natural language understanding, and robotics. ANN consists of interconnected layers of neurons that connect the brain to the brain. The brain is a complex structure that is connected to the brain. The brain is a complex structure that is connected to the brain. ANN consists of interconnected layers of neurons that perform calculations on input data, gradually extracting meaningful features, and generating output predictions. The strength of ANN lies in their ability to learn and adapt through a process called training, where they adjust their internal parameters based on labeled data set. This enables ANN to generalize from examples and make accurate

In [ ]:
ground_truth_transcript = ["Hello, Hello. Hi, Takshi, how are you? Hi, Saurav. I'm good. So today we're going to talk, Yes, yes, continue. Today we're going to talk about artificial neural networks. That is Ann. Ann have become the cornerstone of modern AI applications. Their ability to mimic the human brains information processing has lead to breakthroughs in fields such as computer vision, natural language understanding, and robotics. Ann consists of interconnected layers of neurons that perform calculations on input data, gradually extracting meaningful features and generating output predictions. The strength of Ann lies in their ability to learn and adapt through a process called training, where they adjust their internal parameters based on labeled data set. This enables Ann to generalize from examples and make accurate predictions on unseen data. From self driving cars to medical diagnostics, Ann are transforming industries and paving the way for, Exciting advancements in AI driven technology. That was a wonderful session. Thank you so much.",
                           "Hello, Takshay. Good morning. Hello, Good morning, Sourav. So as last time we were talking about artificial neural networks, we're going to continue with that discussion. OK. So artificial neural networks have revolutionized machine learning and artificial intelligence. Ann's excel at pattern recognition, classification, regression and other complex tasks. During the training process, Ann learn to recognize patterns and make predictions by adjusting the weights and biases associated with connections between neuron. This process, known as backpropagation, involves iteratively feeding the network with input data and comparing its output with desired output. With advances in deep learning, Ann with numerous layers known as deep neural networks, have achieved remarkable success in various domains such as image and speech recognition, natural language processing and autonomous vehicle. The ability of Anns to learn from data and make accurate predictions makes them powerful tool with the potential for significant impact across industries and scientific research. Okay so thank you so much for this session. Bye."]


In [ ]:
# ground_truth_transcript = ["so Mike Racine man here SF Terry there you can't have that new micro machine aircraft carrier playset the dramatically detail trip we trim replica of the real thing that holds 25 micro machines with type II was fantastic features who totally terrific of it is real working hoist runway two cargo arms if a new micro machine aircraft carry placer and micro machine cargo plane playset that holds 15 micro machines with amazingly mini military features opening close nose real working ramps elevator in cargo door better get it before it takes off the new Mike Racine mini cargo plane playset the Micro Machines aircraft carrier cargo plane playset and vehicles each sold separately from Galoob remember if it doesn't say micro machines it's not the real thing",
#                 "this is the micro machine man presenting the most miniature motorcade of micro machines each one has dramatic details terrific trim precision paint jobs plus incredible micro machine pocket place that there's a police station fire station restaurant service station and more perfect pocket portables to take any place and there are many miniature places to play with each one comes with its own special edition micro machine vehicle and fun fantastic features that miraculously move raise the boat lift at the airport marina man the gun turret at the army base clean your car the car wash raise the toll bridge and these place sets fit together to form a micro machine world micro machine pocket place that search reminds me tiny so perfectly precise so dazzlingly detailed you want to pocket them all micro machines are micro machine pocket place that sold separately from galub the smaller they are the better they are"]


In [ ]:
rouge = evaluate.load('rouge')
rouge1 = []
rouge2 = []
rougeL = []
rougeLsum = []

for i in range(len(generated_texts)):
  predictions = [generated_texts[i]]
  references = [ground_truth_transcript[i]]
  results = rouge.compute(predictions=predictions,
                          references=references)
  rouge1.append(results['rouge1'])
  rouge2.append(results['rouge2'])
  rougeL.append(results['rougeL'])
  rougeLsum.append(results['rougeLsum'])

In [ ]:
df = pd.DataFrame({'Generated_transcript':generated_texts,
                   'Ground_truth':ground_truth_transcript,
                   'rouge1':rouge1,
                   'rouge2':rouge2,
                   'rougeL':rougeL,
                   'rougeLsum':rougeLsum})
df

,Generated_transcript,Ground_truth,rouge1,rouge2,rougeL,rougeLsum
0,"Hello. Hello. Hi, Akshay. How are you? I'm good. So today we're going to talk. Yes, yes, continue. Today we're going to talk about artificial neural networks. That is ANN. ANN has become the cornerstone of the world. The ability to mimic the human brain's information processing has lead to breakthroughs in fields such as computer vision, natural language understanding, and robotics. ANN consists of interconnected layers of neurons that connect the brain to the brain. The brain is a complex structure that is connected to the brain. The brain is a complex structure that is connected to the brain. ANN consists of interconnected layers of neurons that perform calculations on input data, gradually extracting meaningful features, and generating output predictions. The strength of ANN lies in their ability to learn and adapt through a process called training, where they adjust their internal parameters based on labeled data set. This enables ANN to generalize from examples and make accurate predictions on unseen data. From self-driving cars to medical diagnostics, ANN are transforming industries and paving the way for exciting advancements in AI-driven technology. That was a wonderful session. Thank you so much. you","Hello, Hello. Hi, Takshi, how are you? Hi, Saurav. I'm good. So today we're going to talk, Yes, yes, continue. Today we're going to talk about artificial neural networks. That is Ann. Ann have become the cornerstone of modern AI applications. Their ability to mimic the human brains information processing has lead to breakthroughs in fields such as computer vision, natural language understanding, and robotics. Ann consists of interconnected layers of neurons that perform calculations on input data, gradually extracting meaningful features and generating output predictions. The strength of Ann lies in their ability to learn and adapt through a process called training, where they adjust their internal parameters based on labeled data set. This enables Ann to generalize from examples and make accurate predictions on unseen data. From self driving cars to medical diagnostics, Ann are transforming industries and paving the way for, Exciting advancements in AI driven technology. That was a wonderful session. Thank you so much.",0.847645,0.818942,0.847645,0.847645
1,"Hello, Takshya. Good morning. Hello. Good morning, Sourav. So, as last time we were talking about artificial neural networks, we're going to continue with that discussion. So, artificial neural networks have revolutionized machine learning and artificial intelligence. ANNs excel at pattern recognition, classification, regression and other complex tasks. During the training process, ANNs learn to recognize patterns and make predictions by adjusting the weights and biases associated with connections between neurons. This process, known as backpropagation, involves iteratively feeding the network with input data and comparing its output with desired output. With advances in deep learning, ANNs with numerous layers known as deep neural networks have achieved remarkable success in various domains such as image and speech recognition, natural language processing and autonomous vehicles. The ability of ANNs to learn from data and make accurate predictions makes them a powerful tool with the potential for significant impact across industries and scientific research. Okay. So, thank you so much for this session. Bye.","Hello, Takshay. Good morning. Hello, Good morning, Sourav. So as last time we were talking about artificial neural networks, we're going to continue with that discussion. OK. So artificial neural networks have revolutionized machine learning and artificial intelligence. Ann's excel at pattern recognition, classification, regression and other complex tasks. During the training process, Ann learn to recognize patterns and make predictions by adjusting the weights and biases associated with con

# Text to context

max token = 1024.

In [ ]:
del model_medium

In [ ]:
# summarizer1 = pipeline("summarization", model="knkarthick/MEETING_SUMMARY")
summarizer1 = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
def get_last_fs_index(tokenized_batch):
    for idx, t in enumerate(reversed(tokenized_batch)):
            if t == ".":
                return idx
    return 0
    
def summarization1(text):

  tokenized = summarizer1.tokenizer.tokenize(text) 
  max_token_length = summarizer1.model.config.d_model
  batches = []
  start_token_idx = 0
  i = 0
  max_token_length = summarizer1.model.config.d_model
  while not start_token_idx>=len(tokenized):
      if len(tokenized)>max_token_length:
          batches.append(tokenized[start_token_idx:start_token_idx+max_token_length])
          idx = get_last_fs_index(batches[i])
          batches[i] = batches[i][:max_token_length-idx]
          i+=1
          start_token_idx+=(max_token_length-idx)    
      else:
          batches.append(tokenized)
          break   
  text_summ_mapping = []
  only_summ = []
  for batch in batches:
      batch_str= summarizer1.tokenizer.convert_tokens_to_string(batch)
      summ = summarizer1(batch_str)
      text_summ_mapping.append((batch_str,summ,))
      only_summ.append(summ)
  return only_summ

In [ ]:
transcript = [get_file_content(path) for path in text_path]

In [ ]:
summaries = [summarization1(text) for text in transcript]
summaries

[[[{'summary_text': "Artificial neural networks (ANN) mimic the human brain's information processing. They have led to breakthroughs in fields such as computer vision, natural language understanding, and robotics. From self-driving cars to medical diagnostics, ANN are paving the way for exciting advancements in AI-driven technology."}]],
 [[{'summary_text': 'Artificial neural networks have revolutionized machine learning and artificial intelligence. ANNs excel at pattern recognition, classification, regression and other complex tasks. The ability of ANNs to learn from data and make accurate predictions makes them a powerful tool with the potential for significant impact across industries and scientific research.'}]]]

In [ ]:
final_summaries = []
for sum in summaries:
  for i in sum:
    for j in i:
      final_summaries.append(j['summary_text'])

In [ ]:
final_summaries

["Artificial neural networks (ANN) mimic the human brain's information processing. They have led to breakthroughs in fields such as computer vision, natural language understanding, and robotics. From self-driving cars to medical diagnostics, ANN are paving the way for exciting advancements in AI-driven technology.",
 'Artificial neural networks have revolutionized machine learning and artificial intelligence. ANNs excel at pattern recognition, classification, regression and other complex tasks. The ability of ANNs to learn from data and make accurate predictions makes them a powerful tool with the potential for significant impact across industries and scientific research.']

In [ ]:
ground_truth_summary = ["Artificial Neural Networks (ANN) have revolutionized AI applications by mimicking human brain processes. ANN's ability to learn, adapt, and make accurate predictions through training has led to breakthroughs in computer vision, natural language understanding, and robotics. They are transforming industries such as self-driving cars and medical diagnostics, paving the way for exciting advancements in AI technology.",
                        "Artificial neural networks (ANNs) have revolutionized machine learning by excelling in pattern recognition and prediction tasks through backpropagation. Deep neural networks (DNNs) with multiple layers have achieved impressive success in various domains. ANNs have the potential for significant impact across industries and scientific research."]

In [ ]:
# ground_truth_summary = [
#                         "Introducing the exciting world of Micro Machines! Experience the thrill of the new Micro Machine aircraft carrier playset, a detailed replica with 25 micro machines and impressive features like a working hoist and cargo arms. Don't miss out on the Micro Machine cargo plane playset either, which holds 15 micro machines and boasts mini military features. Hurry and grab these sets along with the accompanying vehicles to enjoy endless fun. Remember, if it doesn't say Micro Machines, it's not the real deal.",
#                         """This is the Micro Machine Man presenting the most midget miniature motorcade of Micro Machines.Each one has dramatic details, terrific trim, precision paint jobs, plus incredible Micro Machine Pocket play sets, physical police station, fire station, restaurant, service station, and more! Micro Machine Pocket play sets, so tremendously tiny, so perfectly precise, so dazzingly detailed, you'll want to pocket them all!"""
#                       ]

In [ ]:
rouge1 = []
rouge2 = []
rougeL = []
rougeLsum = []

for i in range(len(final_summaries)):
  predictions = [final_summaries[i]]
  references = [ground_truth_summary[i]]
  results = rouge.compute(predictions=predictions,
                          references=references)
  rouge1.append(results['rouge1'])
  rouge2.append(results['rouge2'])
  rougeL.append(results['rougeL'])
  rougeLsum.append(results['rougeLsum'])

df = pd.DataFrame({'Generated_summary':final_summaries,
                   'Ground_truth':ground_truth_summary,
                   'rouge1':rouge1,
                   'rouge2':rouge2,
                   'rougeL':rougeL,
                   'rougeLsum':rougeLsum})
df

,Generated_summary,Ground_truth,rouge1,rouge2,rougeL,rougeLsum
0,"Artificial neural networks (ANN) mimic the human brain's information processing. They have led to breakthroughs in fields such as computer vision, natural language understanding, and robotics. From self-driving cars to medical diagnostics, ANN are paving the way for exciting advancements in AI-driven technology.","Artificial Neural Networks (ANN) have revolutionized AI applications by mimicking human brain processes. ANN's ability to learn, adapt, and make accurate predictions through training has led to breakthroughs in computer vision, natural language understanding, and robotics. They are transforming industries such as self-driving cars and medical diagnostics, paving the way for exciting advancements in AI technology.",0.750000,0.470588,0.615385,0.615385
1,"Artificial neural networks have revolutionized machine learning and artificial intelligence. ANNs excel at pattern recognition, classification, regression and other complex tasks. The ability of ANNs to learn from data and make accurate predictions makes them a powerful tool with the potential for significant impact across industries and scientific research.",Artificial neural networks (ANNs) have revolutionized machine learning by excelling in pattern recognition and prediction tasks through backpropagation. Deep neural networks (DNNs) with multiple layers have achieved impressive success in various domains. ANNs have the potential for significant impact across industries and scientific research.,0.516129,0.329670,0.473118,0.473118


In [ ]:
del summarizer1

In [ ]:
# def get_file_content(filepath):
#     filepath = Path(filepath)
#     file = open(filepath,"r")
#     return file.read().strip()

In [ ]:
# def summarize(FileContent,checkpoint,model,tokenizer):
#     length = 0
#     chunk = ""
#     chunks = []
#     count = -1
#     sentences = nltk.tokenize.sent_tokenize(FileContent)
#     for sentence in sentences:
#         count += 1
#         combined_length = len(tokenizer.tokenize(sentence)) + length

#         if combined_length <= tokenizer.max_len_single_sentence-1:
#             chunk += sentence + " "
#             length = combined_length

#             if count == len(sentences) - 1:
#                 chunks.append(chunk.strip())

#         else:
#             chunks.append(chunk.strip())

#             length = 0
#             chunk = ""
#             chunk += sentence + " "
#             length = len(tokenizer.tokenize(sentence))
#     inputs = [tokenizer(chunk,return_tensors='pt') for chunk in chunks]
#     print(f"Processing {len(chunks)} chunks")
#     summ = []
#     for ip in tqdm(inputs):
#         output = model.generate(**ip)
#         summ.append(tokenizer.decode(*output,skip_special_tokens=True))
#     return ' '.join(summ)

In [ ]:
# def generate_summaries(path_to_text,save_path='/content/summary/'):
#     model_name = int(input("Press 1: facebook/bart-large-cnn \nPress 2: sshleifer/distilbart-cnn-12-6: "))
#     if model_name == 1:
#         checkpoint = "facebook/bart-large-cnn"
#     elif model_name == 2:
#         checkpoint = "sshleifer/distilbart-cnn-12-6"
#     else:
#         print("Please enter a valid number")
     
#     model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#     tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#     if type(path_to_text) == str:
#         path_to_text = [path_to_text]
#     elif type(path_to_text) == list:
#         pass
#     else:
#         print("Invalid type")

#     generated_summaries = []
#     try:
#       for path in path_to_text: 
#           text = get_file_content(path)
#           name = save_path+path.split('/')[-1]
#           path_for_writing = Path(name)
#           checkpoint_name = checkpoint.replace('/','_')
#           checkpoint_name = checkpoint_name.replace('-','_')
#           with open(f'{path_for_writing}_{checkpoint_name}', 'w') as f:
#               summary = summarize(text,checkpoint,model,tokenizer)
#               f.write(summary)
#     except:
#       del tokenizer
#       del model
#     else: 
#       del tokenizer
#       del model
#     finally:
#       del tokenizer
#       del model

In [ ]:
# generate_summaries(text_path)

In [ ]:
# texts = os.listdir('/content/texts')
# texts = ['/content/texts/'+text for text in texts]
# texts

In [ ]:
from huggingface_hub import from_pretrained_keras
import numpy as np
import gradio as gr
import transformers
import tensorflow as tf

class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data."""
    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=32,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=128,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

model = from_pretrained_keras("keras-io/bert-semantic-similarity")
labels = ["contradiction", "entailment", "neutral"]

def predict(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    probs = model.predict(test_data[0])[0]
    
    labels_probs = {labels[i]: float(probs[i]) for i, _ in enumerate(labels)}
    return labels_probs
    
    #idx = np.argmax(proba)
    #proba = f"{proba[idx]*100:.2f}%"
    #pred = labels[idx]
    #return f'The semantic similarity of two input sentences is {pred} with {proba} of probability'

inputs = [
         gr.Audio(source = "upload", label='Upload audio file', type="filepath"),
]

examples = [["Two women are observing something together.", "Two women are standing with their eyes closed."],
            ["A smiling costumed woman is holding an umbrella", "A happy woman in a fairy costume holds an umbrella"],
            ["A soccer game with multiple males playing", "Some men are playing a sport"],            
]

config.json not found in HuggingFace Hub.


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
gr.Interface(
    fn=predict,
    title="Semantic Similarity with BERT",
    description = "Natural Language Inference by fine-tuning BERT model on SNLI Corpus 📰",
    inputs=["text", "text"],
    examples=examples,
    #outputs=gr.Textbox(label='Prediction'),
    outputs=gr.outputs.Label(num_top_classes=3, label='Semantic similarity'),
    cache_examples=True,
    article = "Author: <a href=\"https://huggingface.co/vumichien\">Vu Minh Chien</a>. Based on the keras example from <a href=\"https://keras.io/examples/nlp/semantic_similarity_with_bert/\">Mohamad Merchant</a>",
).launch(debug=True, enable_queue=True)

  warnings.warn(

  super().__init__(num_top_classes=num_top_classes, type=type, label=label)



Caching examples at: '/content/gradio_cached_examples/17'


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
  warnings.warn(



1/1 [==============================] - 3s 3s/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 62ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 63ms/step
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://98f4011de5b819da20.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
  warnings.warn(



1/1 [==============================] - 0s 88ms/step
